In [12]:
import os
import json

# 设置数据根目录路径
# 假设脚本在仓库根目录下运行，数据在 assets/data
ROOT_DIR = os.path.join("../assets", "data")

def get_max_size(puzzles_dict):
    """
    遍历 puzzles 字典，解析第一行尺寸，返回最大尺寸字符串(按面积比较)。
    如果无法获取，返回 "-"
    """
    max_area = -1
    max_size_str = "-"

    for key, val in puzzles_dict.items():
        if not isinstance(val, dict):
            continue
            
        problem_str = val.get("problem", "")
        if not problem_str:
            continue

        # 获取第一行
        first_line = problem_str.strip().split('\n')[0].strip()
        tokens = first_line.split()

        # 尝试读取前两个数字
        if len(tokens) >= 2:
            try:
                # 假设格式为 "Width Height" 或 "Rows Cols"
                # 这里只作为数字解析计算面积，不做具体宽高定义的区分
                dim1 = int(tokens[0])
                dim2 = int(tokens[1])
                area = dim1 * dim2
                
                if area > max_area:
                    max_area = area
                    max_size_str = f"{dim1}x{dim2}"
            except ValueError:
                continue
    
    return max_size_str

def generate_markdown_table():
    if not os.path.exists(ROOT_DIR):
        print(f"Error: Directory '{ROOT_DIR}' not found.")
        return

    # 获取所有子文件夹并排序
    subdirs = [d for d in os.listdir(ROOT_DIR) if os.path.isdir(os.path.join(ROOT_DIR, d))]
    subdirs.sort() # 按字母顺序排序

    table_data = [] # 存储每一行的数据
    total_problems = 0
    total_solutions = 0

    # 表头
    headers = ["No.", "Puzzle Name", "Problems", "Solutions", "Max Size"]
    
    # 遍历每个谜题文件夹
    for idx, puzzle_name in enumerate(subdirs, 1):
        puzzle_dir = os.path.join(ROOT_DIR, puzzle_name)
        
        # 构建文件路径
        prob_json_path = os.path.join(puzzle_dir, "problems", f"{puzzle_name}_puzzles.json")
        sol_json_path = os.path.join(puzzle_dir, "solutions", f"{puzzle_name}_solutions.json")

        # 初始化当前行的变量
        p_count = "-"
        s_count = "-"
        max_size = "-"

        # 1. 处理 Problems JSON
        if os.path.exists(prob_json_path):
            try:
                with open(prob_json_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    # 统计 puzzle 数量 (基于字典 keys 数量，比 info count 更准确)
                    puzzles = data.get("puzzles", {})
                    count = len(puzzles)
                    p_count = count
                    total_problems += count
                    try:
                    # 计算最大尺寸
                        max_size = get_max_size(puzzles)
                    except Exception as e:
                        # 解析出错保持 "-"
                        pass
            except Exception as e:
                # 解析出错保持 "-"
                
                pass

        # 2. 处理 Solutions JSON
        if os.path.exists(sol_json_path):
            try:
                with open(sol_json_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    # 统计 solution 数量
                    solutions = data.get("solutions", {})
                    count = len(solutions)
                    s_count = count
                    total_solutions += count
            except Exception as e:
                pass

        table_data.append([str(idx), puzzle_name, str(p_count), str(s_count), max_size])

    # --- 生成 Markdown 输出 ---

    # 打印表头
    print(f"| {' | '.join(headers)} |")
    print(f"| {' | '.join(['---'] * len(headers))} |")

    # 打印数据行
    for row in table_data:
        print(f"| {' | '.join(row)} |")

    # 打印统计行
    # No. 为空, Name 为 Total, Counts 加粗, Size 为空
    print(f"| | **Total** | **{total_problems}** | **{total_solutions}** | - |")

if __name__ == "__main__":
    generate_markdown_table()

| No. | Puzzle Name | Problems | Solutions | Max Size |
| --- | --- | --- | --- | --- |
| 1 | ABCEndView | 0 | 0 | - |
| 2 | Akari | 970 | 970 | 100x100 |
| 3 | Battleship | - | - | - |
| 4 | Binairo | 380 | 380 | 14x14 |
| 5 | Buraitoraito | 101 | 100 | 15x15 |
| 6 | ButterflySudoku | 77 | 77 | 12x12 |
| 7 | Clueless1Sudoku | 29 | 29 | 27x27 |
| 8 | Clueless2Sudoku | 40 | 40 | 27x27 |
| 9 | Creek | 440 | 440 | 40x50 |
| 10 | Dominos | 580 | 579 | 10x11 |
| 11 | DoubleMinesweeper | - | - | - |
| 12 | EntryExit | 170 | 170 | 16x16 |
| 13 | Eulero | 148 | 148 | 5x5 |
| 14 | EvenOddSudoku | 129 | 129 | 9x9 |
| 15 | Fillomino | 840 | 840 | 50x64 |
| 16 | Fivecells | - | - | - |
| 17 | Fobidoshi | - | - | - |
| 18 | Foseruzu | - | - | - |
| 19 | Fuzuli | 160 | 160 | 8x8 |
| 20 | Gappy | 429 | 427 | 18x18 |
| 21 | Gattai8Sudoku | 120 | 120 | 21x33 |
| 22 | GokigenNaname | - | - | - |
| 23 | GrandTour | 350 | 350 | 15x15 |
| 24 | Hakyuu | 480 | 480 | 30x45 |
| 25 | Heyawake | - | - | - |
| 26

| No. | Puzzle Name | Problems | Solutions | Max Size |
| --- | --- | --- | --- | --- |
| 1 | ABCEndView | 0 | 0 | - |
| 2 | Akari | 970 | 970 | 100x100 |
| 3 | Battleship | - | - | - |
| 4 | Binairo | 380 | 380 | 14x14 |
| 5 | Buraitoraito | 101 | 100 | 15x15 |
| 6 | ButterflySudoku | 77 | 77 | 12x12 |
| 7 | Clueless1Sudoku | 29 | 29 | 27x27 |
| 8 | Clueless2Sudoku | 40 | 40 | 27x27 |
| 9 | Creek | 440 | 440 | 40x50 |
| 10 | Dominos | 580 | 579 | 10x11 |
| 11 | DoubleMinesweeper | - | - | - |
| 12 | EntryExit | 170 | 170 | 16x16 |
| 13 | Eulero | 148 | 148 | 5x5 |
| 14 | EvenOddSudoku | 129 | 129 | 9x9 |
| 15 | Fillomino | 840 | 840 | 50x64 |
| 16 | Fivecells | - | - | - |
| 17 | Fobidoshi | - | - | - |
| 18 | Foseruzu | - | - | - |
| 19 | Fuzuli | 160 | 160 | 8x8 |
| 20 | Gappy | 429 | 427 | 18x18 |
| 21 | Gattai8Sudoku | 120 | 120 | 21x33 |
| 22 | GokigenNaname | - | - | - |
| 23 | GrandTour | 350 | 350 | 15x15 |
| 24 | Hakyuu | 480 | 480 | 30x45 |
| 25 | Heyawake | - | - | - |
| 26 | Hitori | - | - | - |
| 27 | JigsawSudoku | 680 | 680 | 9x9 |
| 28 | Kakkuru | 400 | 400 | 9x9 |
| 29 | Kakurasu | 280 | 280 | 11x11 |
| 30 | Kakuro | 999 | 999 | 31x46 |
| 31 | KillerSudoku | 810 | 810 | 9x9 |
| 32 | Kuroshuto | - | - | - |
| 33 | LITS | 419 | 419 | 40x57 |
| 34 | Magnetic | 439 | 439 | 12x12 |
| 35 | Makaro | 190 | 190 | 15x15 |
| 36 | Maze-a-pix | - | - | - |
| 37 | Minesweeper | 360 | 360 | 14x24 |
| 38 | Mosaic | 165 | 104 | 118x100 |
| 39 | Munraito | 360 | 360 | 12x12 |
| 40 | Nanbaboru | 270 | 270 | 9x9 |
| 41 | Nondango | 110 | 110 | 14x14 |
| 42 | Nonogram | 2340 | 2339 | 30x40 |
| 43 | Norinori | 289 | 289 | 36x54 |
| 44 | OneToX | - | - | - |
| 45 | Patchwork | 211 | 211 | 12x12 |
| 46 | Pfeilzahlen | 360 | 360 | 8x8 |
| 47 | Pills | 164 | 163 | 10x10 |
| 48 | Polyiamond | - | - | - |
| 49 | Polyminoes | - | - | - |
| 50 | Renban | 150 | 150 | 9x9 |
| 51 | SamuraiSudoku | 272 | 272 | 21x21 |
| 52 | Shikaku | 500 | 500 | 31x45 |
| 53 | ShogunSudoku | 90 | 90 | 21x45 |
| 54 | Simpleloop | 70 | 70 | 17x18 |
| 55 | Slitherlink | 1176 | 1153 | 60x60 |
| 56 | Snake | 230 | 230 | 12x12 |
| 57 | SoheiSudoku | 120 | 120 | 21x21 |
| 58 | SquareO | 120 | 80 | 15x15 |
| 59 | Starbattle | 307 | 307 | 15x15 |
| 60 | Sternenhimmel | 29 | 29 | 17x17 |
| 61 | Str8t | 560 | 560 | 9x9 |
| 62 | Sudoku | 125 | 125 | 16x16 |
| 63 | Suguru | 200 | 200 | 10x10 |
| 64 | SumoSudoku | 110 | 110 | 33x33 |
| 65 | Tatamibari | 150 | 150 | 14x14 |
| 66 | TennerGrid | 375 | 374 | 6x10 |
| 67 | Tent | 706 | 706 | 30x30 |
| 68 | TerraX | 80 | 80 | 17x17 |
| 69 | Thermometer | 250 | 250 | 10x10 |
| 70 | TilePaint | 109 | 109 | 16x16 |
| 71 | WindmillSudoku | 150 | 150 | 21x21 |
| 72 | Yajilin | 610 | 610 | 39x57 |
| 73 | YinYang | 170 | 170 | 14x14 |
| 74 | monotone | - | - | - |
| | **Total** | **20948** | **20817** | - |
